In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import visuals as vs
import datetime
import csv
import xgboost as xgb
import itertools
import os
import operator
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import r2_score,mean_squared_error
from sklearn import grid_search
%matplotlib inline
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.base import TransformerMixin
from sklearn import cross_validation
from matplotlib import pylab as plt
plot = True
goal = 'Sales'
myid = 'Id'
from sklearn.cross_validation import train_test_split
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

/home/sampson/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [34]:
def get_data():  ## 数据获取函数
    store = pd.read_csv('./store.csv')
    train_org = pd.read_csv('./train.csv',dtype={'StateHoliday':pd.np.string_})
    test_org = pd.read_csv('./test.csv',dtype={'StateHoliday':pd.np.string_})
    train = pd.merge(train_org,store,on='Store',how='left')
    test = pd.merge(test_org,store,on='Store',how='left')
    
    features1 = ['Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday', 
                 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 
                 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 
                 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

    feature_non = ['Promo','Store','Date', 'StateHoliday','SchoolHoliday', 'StoreType', 'Assortment', 'Promo2', 'PromoInterval']
    return (train,test,features1,feature_non)

In [31]:
def process_data(train,test,features,features_no): ##数据处理
    train = train[train['Open']==1]
    
    test = test[test['Open']==1]
    for rossm in [train,test]:
        rossm['year'] = rossm.Date.apply(lambda x: int(x.split('-')[0]))
        rossm['month']= rossm.Date.apply(lambda x: int(x.split('-')[1]))
        rossm['day']  = rossm.Date.apply(lambda x: int(x.split('-')[2]))

        rossm['promojan'] = rossm.PromoInterval.apply(lambda x:0 if isinstance(x,float) else 1 if 'Jan' in x else 0)
        rossm['promofeb'] = rossm.PromoInterval.apply(lambda x:0 if isinstance(x,float) else 1 if 'Feb' in x else 0)
        rossm['promomar'] = rossm.PromoInterval.apply(lambda x:0 if isinstance(x,float) else 1 if 'Mar' in x else 0)
        rossm['promoapr'] = rossm.PromoInterval.apply(lambda x:0 if isinstance(x,float) else 1 if 'Apr' in x else 0)
        rossm['promomay'] = rossm.PromoInterval.apply(lambda x:0 if isinstance(x,float) else 1 if 'May' in x else 0)
        rossm['promojun'] = rossm.PromoInterval.apply(lambda x:0 if isinstance(x,float) else 1 if 'Jun' in x else 0)
        rossm['promojul'] = rossm.PromoInterval.apply(lambda x:0 if isinstance(x,float) else 1 if 'Jul' in x else 0)
        rossm['promoaug'] = rossm.PromoInterval.apply(lambda x:0 if isinstance(x,float) else 1 if 'Aug' in x else 0)
        rossm['promosep'] = rossm.PromoInterval.apply(lambda x:0 if isinstance(x,float) else 1 if 'Sep' in x else 0)
        rossm['promooct'] = rossm.PromoInterval.apply(lambda x:0 if isinstance(x,float) else 1 if 'Oct' in x else 0)
        rossm['promonov'] = rossm.PromoInterval.apply(lambda x:0 if isinstance(x,float) else 1 if 'Nov' in x else 0)
        rossm['promodec'] = rossm.PromoInterval.apply(lambda x:0 if isinstance(x,float) else 1 if 'Dec' in x else 0)
    
    day_dummies = pd.get_dummies(train['DayOfWeek'],prefix='Day')
    day_dummies.drop(['Day_7'],axis=1,inplace=True)  #删除周日的数据
    train = train.join(day_dummies)

    
    day_dummies_test = pd.get_dummies(test['DayOfWeek'],prefix='Day')
    day_dummies_test.drop(['Day_7'],axis=1,inplace=True)  #删除周日数据
    test = test.join(day_dummies_test)
    
    noisy_features = [myid,'Date']
    features = [c for c in features if c not in noisy_features]
    features_non_numeric = [c for c in features_no if c not in noisy_features]
    features.extend(['year','month','day'])
    class DataFrameInputer(TransformerMixin):
        def __init__(self):
                """
                """
        def fit(self,X,y=None):
            self.fill = pd.Series([X[c].value_counts().index[0] 
                                  if X[c].dtype==np.dtype('O') 
                                   else X[c].mean() for c in X],index = X.columns)
            return self
        def transform(self,X,y=None):
            return X.fillna(self.fill)
        
    train = DataFrameInputer().fit_transform(train)
    test  = DataFrameInputer().fit_transform(test)
        
    le = LabelEncoder()
    for col in features:
        le.fit(list(train[col])+list(test[col]))
        train[col] = le.transform(train[col])
        test[col] = le.transform(test[col])
        
    scaler = StandardScaler()
    for col in set(features) - set(features_no) - set([]):
        try:
            scaler.fit(list(train[col])+list(test[col]))
        except:
            print(col)
        train[col] = scaler.transform(train[col])
        test[col] = scaler.transform(test[col])
    return (train,test,features,features_no)

In [35]:
train,test,features,features_non_numeric = get_data()
train,test,features,features_non_numeric = process_data(train,test,features,features_non_numeric)
# train.drop(['Open','Date','DayOfWeek'],axis=1,inplace=True)
# test.drop(['Open','Date','DayOfWeek'],axis=1,inplace=True)
rossm_train = dict(list(train.groupby('Store')))
rossm_test = dict(list(test.groupby('Store')))

In [12]:
train,test,features,features_non_numeric = get_data()
train,test,features,features_non_numeric = process_data(train,test,features,features_non_numeric)
a1 = train.columns
a2 = ['Date','Sales','Customers','DayOfWeek']
a1 = [f for f in a1 if f not in a2]
# XGB_native(train,test,a1,features_non_numeric)
rossm_train = dict(list(train.groupby('Store')))
rossm_test = dict(list(test.groupby('Store')))

### RandomForest 建模

In [37]:
store = train
y = store['Sales']
x = store.drop(['Sales','Store','Customers','Date'],axis=1)
X_train,X_test,y_train,y_test = cross_validation.train_test_split(x,y,test_size=0.3)

clf = RandomForestRegressor(n_estimators=70,max_features='sqrt',max_depth=13)
clf.fit(X_train,y_train)
print clf.score
print r2_score(clf.predict(X_test),y_test)

<bound method RandomForestRegressor.score of RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=13,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=70, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)>
-0.704077194552


###  RandomForest 参数调优

In [38]:
clf_b = RandomForestRegressor()
paramer = {'n_estimators':[10,20],'max_features':['auto','sqrt'],
           'max_depth':[7,15,20],"bootstrap": [True, False]}
gs = grid_search.GridSearchCV(clf_b,paramer,n_jobs=-1,cv=2)
gs.fit(X_train,y_train)

print r2_score(gs.predict(X_test),y_test)

0.821572935275


In [40]:
store = rossm_train[8]
y = store['Sales']
x = store.drop(['Sales','Store','Customers','Date'],axis=1)
X_train,X_test,y_train,y_test = cross_validation.train_test_split(x,y,test_size=0.3)
clf_b = RandomForestRegressor()
paramer = {'n_estimators':[10,20,30,40,50,60,70,80,90,100],'max_features':['auto','sqrt'],
           'max_depth':range(6,15),"bootstrap": [True, False]}
gs = grid_search.GridSearchCV(clf_b,paramer,n_jobs=-1,cv=2)
gs.fit(X_train,y_train)
print gs.best_params_
print r2_score(gs.predict(X_test),y_test)

{'max_features': 'auto', 'n_estimators': 90, 'bootstrap': True, 'max_depth': 11}
0.70088964412


In [41]:
gs.best_score_

0.6930088444242578

In [42]:
random1 = []
xgb1 = []

random2 = []
xgb2 = []

In [43]:
for i in rossm_test:
    store = rossm_train[i]
    y = store['Sales']
    x = store.drop(['Sales','Store','Customers','Date'],axis=1)
    X_train,X_test,y_train,y_test = cross_validation.train_test_split(x,y,test_size=0.3)
    
    clf_b = RandomForestRegressor()
    paramer = {'n_estimators':[60,70,80],'max_features':['auto'],
               'max_depth':range(10,13),"bootstrap": [True]}
    gs = grid_search.GridSearchCV(clf_b,paramer,n_jobs=-1,cv=2)
    gs.fit(X_train,y_train)
    random1.append(gs.best_score_)
    random2.append(r2_score(gs.predict(X_test),y_test))


In [44]:
print np.mean(random1),np.mean(random2)

0.665965750294 0.589146638467



##### RandomForestRegressor最优参数的结果0.589146638467没有xgboost算法没有调惨的0.626765270346分数高，
##### 最终选择xgboost算法为最终建模算法